# Bài Test

Bài 1: Viết một chương trình nhập vào một file văn bản bằng tiếng anh và thống kê số lần xuất hiện của các từ trong văn bản đó

In [1]:
from nltk import word_tokenize
from collections import Counter
from itertools import chain

def word_frequency(file_path):
    """ Read file, where each sentence is dilineated by a `\n`.
    @param file_path (str): path to file containing corpus
    @return dictionary contains word frequency
    """
    data = []
    for line in open(file_path):
        sent = word_tokenize(line.strip().lower())
        data.append(sent)

    word_freq = Counter(chain(*data))
    return dict(word_freq)

Thử hàm với file toy_dataset.txt

In [2]:
word_frequency("toy_dataset.txt")

{'i': 2,
 'do': 1,
 "n't": 1,
 'want': 1,
 'to': 3,
 'go': 2,
 'school': 1,
 'have': 1}

Bài 2: Cho dạng ngày tháng: YYYY-mm-dd. Viết biểu thức chính quy kiểm tra một dãy ký tự có phải là dạng ngày tháng năm đã cho ở trên hay không

In [3]:
import re
def check_form(date):
    """
    @param date (str): string to check
    @return boolean: True if date is in the correct form or False if not
    """
    pattern = re.compile(r"^\d{4}-(02-(0[1-9]|[12][0-9])|(0[469]|11)-(0[1-9]|[12][0-9]|30)|(0[13578]|1[02])-(0[1-9]|[12][0-9]|3[01]))$")
    if pattern.search(date):
        return True
    else:
        return False

In [4]:
check_form("2020-04-31")

False

In [5]:
check_form("2020-04-30")

True

Bài 3: 
3.1 Đọc và phân chia dữ liệu thành 3 tập train.txt, dev.txt, test.txt với tỉ lệ 7:1:2, nhãn 1 và 0 cân bằng trên cả 3 hệ 

In [6]:
# Đọc file dataset
data = []
label = []
for line in open("dataset.txt"):
    x = line.strip().lower()
    label.append(int(x[-1]))
    data.append(x[:-1].strip())

Chia dữ liệu thành 2 phần train và test 

In [7]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state = 1, stratify=label)

Rồi lại chia tập train thành 2 phần là train và develop

In [8]:
x_train, x_dev, y_train, y_dev = train_test_split(X_train, Y_train, test_size = 0.125, random_state = 1, stratify=Y_train)

Lưu dữ liệu vào file

In [9]:
# train
f = open('train.txt', 'w')

for i in range(len(x_train)):
    f.write(x_train[i] + "\t" + str(y_train[i]) + "\n")
f.close()

# dev
f = open('dev.txt', 'w')

for i in range(len(x_dev)):
    f.write(x_dev[i] + "\t" + str(y_dev[i]) + "\n")
f.close()

# test
f = open('test.txt', 'w')

for i in range(len(x_test)):
    f.write(x_test[i] + "\t" + str(y_test[i]) + "\n")
f.close()


In [10]:
import torch
X_train = [word_tokenize(seq) for seq in x_train]
x_dev = [word_tokenize(seq) for seq in x_dev]
x_test = [word_tokenize(seq) for seq in x_test]
y_dev = torch.LongTensor(y_dev)
y_test = torch.LongTensor(y_test)

 3.2 Xây dựng một model từ tập train và dev rồi sau đó eval dựa trên tập test


In [11]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import torch.nn as nn

from model import RNNModel, RNNClassifierModel
from vocab import Vocab

# xây dựng vocab dựa vào tập train
vocab = Vocab.from_corpus(X_train, size = 300, freq_cutoff=5)

# xây dựng mạng RNN sử dụng LSTM
rnn = RNNModel(len(vocab),rnn_cell_class=nn.LSTM, bidirectional=True, embed_dim=40, hidden_dim=60)

# Xây dựng model
RNN = RNNClassifierModel(rnn, output_dim = 2, classifier_activation = nn.Tanh(), vocab = vocab)

number of word types: 1651, number of word types w/ frequency >= 5: 273


In [12]:
from torch.utils.data import DataLoader

def length(data):
    return torch.LongTensor([len(seq) for seq in data])

def collate_fn(batch):
    data,label = zip(*batch)
    y = torch.LongTensor(label)
    x = [word_tokenize(seq) for seq in data]
    return x, y

batch_size = 32
learning_rate = 0.0001
lr_decay = 0.5
epochs = 50
model_save_path = "model.bin"
patience = 0

data = list(zip(x_train, y_train))
loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

optimizer = torch.optim.Adam(RNN.parameters(), lr=learning_rate)
lossfunction = nn.CrossEntropyLoss()

for epoch in range(epochs):
    RNN.train()
    total_loss = 0
    for x, y in loader:
        
        optimizer.zero_grad()
        
        y_pred = RNN(x, length(x)) 
        
        loss = lossfunction(y_pred, y)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss
    
    # Early stop and decay lr
    if epoch %2 ==0:
        print("Loss: {:.2f}, epoch: {}".format(total_loss, epoch))
        RNN.eval()
        with torch.no_grad():
            y_dev_pre = RNN.predict(x_dev, length(x_dev))
            score = f1_score(y_dev, y_dev_pre, average = "macro")
        if epoch == 0:
            best_score = score
        if score> best_score:
            patience = 0
            best_score = score
            print("Save model with dev_f1_score = {:.2f}".format(best_score))
            RNN.save(model_save_path)
            
        else:
            patience += 1
        if patience == 5:
            # decay lr, and restore from previously best checkpoint
            lr = optimizer.param_groups[0]['lr'] * lr_decay
            print('Load previously best model and decay learning rate to %f' % lr)
            RNN = RNNClassifierModel.load(model_save_path)
            
            optimizer = torch.optim.Adam(RNN.parameters(), lr=lr)
            
            # reset patience
            patience = 0

Loss: 15.19, epoch: 0
Loss: 15.06, epoch: 2
Save model with dev_f1_score = 0.60
save model parameters to [model.bin]
Loss: 14.94, epoch: 4
Loss: 14.80, epoch: 6
Save model with dev_f1_score = 0.61
save model parameters to [model.bin]
Loss: 14.63, epoch: 8
Loss: 14.40, epoch: 10
Loss: 14.08, epoch: 12
Loss: 13.62, epoch: 14
Save model with dev_f1_score = 0.63
save model parameters to [model.bin]
Loss: 13.05, epoch: 16
Loss: 12.19, epoch: 18
Loss: 11.31, epoch: 20
Loss: 10.56, epoch: 22
Loss: 9.96, epoch: 24
Save model with dev_f1_score = 0.66
save model parameters to [model.bin]
Loss: 9.23, epoch: 26
Loss: 8.70, epoch: 28
Save model with dev_f1_score = 0.67
save model parameters to [model.bin]
Loss: 8.12, epoch: 30
Save model with dev_f1_score = 0.68
save model parameters to [model.bin]
Loss: 7.62, epoch: 32
Loss: 7.30, epoch: 34
Save model with dev_f1_score = 0.69
save model parameters to [model.bin]
Loss: 6.68, epoch: 36
Loss: 6.33, epoch: 38
Loss: 5.87, epoch: 40
Loss: 5.92, epoch: 4

Đánh giá model dựa vào tập test

In [13]:
RNN = RNNClassifierModel.load(model_save_path)
y_pred = RNN.predict(x_test, length(x_test))
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.60      0.60       100
           1       0.60      0.61      0.61       100

    accuracy                           0.60       200
   macro avg       0.61      0.60      0.60       200
weighted avg       0.61      0.60      0.60       200



Ta sẽ đánh giá model trên dựa vào giá trị của "Macro F1 Score". Kết quả cũng có thể tạm chấp nhận được vì model mà ta xây dựng khá là đơn giản và tập dữ liệu training chỉ có 700 sample - 1 tập dữ liệu rất bé.
Để cải thiện bài toán, ta cần phải:
- Thu thập nhiều dữ liệu training hơn
- Sau khi có đủ lượng dữ liệu training cần thiết, sử dụng Deep Bidirectional LSTM thay vì chỉ 1 layer Bidirectional LSTM như bài làm trên. Hoặc có thể sử dụng cách tiếp cận khác: Fine-tuning Bert Pretrained để xem kết quả có cả thiện hơn so với các cách tiếp cận trước đó hay không
- Bài làm trên sử dụng "last_hidden_state" để predict label, ta cũng có thể có hướng tiếp cận khác: Thay vì sử dụng "last_hidden_state", chúng ta cộng tổng và lấy trung bình của các "hidden_state"
- HyperParameter Search cũng là một trong những cách để cải thiện kết quả


3.3 Xây dựng một chương trình tương tác cho phép nhập một câu từ dòng lệnh và dự đoán ra nhãn 1 hay 0 cho câu nhập

In [14]:
from model import RNNClassifierModel
import torch
from nltk import word_tokenize

def deploy():
    RNN = RNNClassifierModel.load("model.bin")
    print("Sentiment Analysis: 1-positive, 0-negative")
    seq = input("Điền câu để kiểm tra:")
    x = [word_tokenize(seq)]
    l = torch.LongTensor([len(x[0])])
    print(RNN.predict(x,l).item())
deploy()

Sentiment Analysis: 1-positive, 0-negative
Điền câu để kiểm tra:The food is good
1
